In [1]:
import numpy as np

In [50]:
class RegModel:
    def __init__(self, data: np.array, labels: np.array, learning_rate = 0.1, set_divide=(0.7, 0.2, 0.1)):
        assert len(labels) == len(data), 'Sizes of Labels and Train Data must match'
        p = np.random.permutation(len(data))
        data = data[p]
        labels = labels[p]
        
        # Normalize Data
        data = (data - data.mean()) / data.std()
        labels = (labels - labels.mean()) / labels.std()
        
        self.num_vars = data.shape[1]
        self.l = learning_rate
        
        self.train_set = data[0: int(set_divide[0] * len(data))]
        self.train_labels = labels[0: int(set_divide[0] * len(data))]
        
        self.validation_set = data[int(set_divide[0] * len(data)): int((set_divide[0] + set_divide[1]) * len(data))]
        self.validation_labels = labels[int(set_divide[0] * len(data)): int((set_divide[0] + set_divide[1]) * len(data))]
        
        self.test_set = data[int((set_divide[0] + set_divide[1]) * len(data)):]
        self.test_labels = labels[int((set_divide[0] + set_divide[1]) * len(data)):]
        
        self.initialize_weights()
        
    def initialize_weights(self, feature_engineering=False):
        if feature_engineering:
            # Feature Engineering
            pass
        self.weights = np.random.normal(size=(self.num_vars, 1))
        self.bias = np.zeros((1,1))
    
    @staticmethod
    def _get_loss(results, labels):
        return np.sum(np.square(results - labels)) / len(results)
    
    def _run_model(self, data):
        return np.dot(data, self.weights) + self.bias
    
    def _update_weights(self, predicted, labels, inputs):
        self.weights -= self.l * (2 / len(labels) * np.sum(inputs * (predicted - labels)))
        self.bias -= self.l * (2/len(labels) * np.sum(predicted - labels))
        
    def epoch(self, num_epochs):
        for i in range(num_epochs):
            pred = self._run_model(self.train_set)
            loss = self._get_loss(pred, self.train_labels)
            
            print(f'[+] Epoch {i}: Loss = {round(loss, 2)}')
            self._update_weights(pred, self.train_labels, self.train_set)
    
    def test_model(self):
        pred = self._run_model(self.test_set)
        loss = self._get_loss(pred, self.test_labels)
        
        print(f'[+] Loss in Test Set = {round(loss, 2)}')
    
    def predict(self, inputs):
        return self._run_model(inputs)

In [28]:
data = np.load('ProcessedData/WeatherConditions/train_data.npy')
labels = np.load('ProcessedData/WeatherConditions/labels.npy')
labels = np.reshape(labels, (labels.shape[0], 1))

In [52]:
model = RegModel(data, labels)

In [53]:
model.epoch(10)

[+] Epoch 0: Loss = 1.13
[+] Epoch 1: Loss = 1.03
[+] Epoch 2: Loss = 1.02
[+] Epoch 3: Loss = 1.02
[+] Epoch 4: Loss = 1.02
[+] Epoch 5: Loss = 1.02
[+] Epoch 6: Loss = 1.02
[+] Epoch 7: Loss = 1.02
[+] Epoch 8: Loss = 1.02
[+] Epoch 9: Loss = 1.02


In [54]:
model.test_model()

[+] Loss in Test Set = 1.02
